In [1]:
from pandas import read_csv, Series, concat
from numpy import array
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.losses import categorical_crossentropy
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
train_df = read_csv('../input/train.csv')
test_df = read_csv('../input/test.csv')

In [3]:
train_df.shape

(42000, 785)

In [4]:
test_df.shape

(28000, 784)

In [5]:
x_train = train_df.iloc[:,1:785]
x_train.shape

(42000, 784)

In [6]:
y_train = train_df.iloc[:,0]
y_train.shape

(42000,)

In [7]:
x_test = test_df
x_test.shape

(28000, 784)

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=0.33, random_state=42)

In [9]:
print("x_train : ",x_train.shape)
print("x_valid : ",x_valid.shape)
print("x_test : ",x_test.shape)
print("y_train : ",y_train.shape)
print("y_valid : ",y_valid.shape)

x_train :  (28140, 784)
x_valid :  (13860, 784)
x_test :  (28000, 784)
y_train :  (28140,)
y_valid :  (13860,)


In [10]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_valid = x_valid.astype('float32')
x_train /= 255
x_test /= 255
x_valid /= 255

In [11]:
x_train = array(x_train)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_train.shape

(28140, 28, 28, 1)

In [12]:
x_valid = array(x_valid)
x_valid = x_valid.reshape(x_valid.shape[0], 28, 28, 1)
x_valid.shape

(13860, 28, 28, 1)

In [13]:
x_test = array(x_test)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test.shape

(28000, 28, 28, 1)

In [14]:
y_train = to_categorical(y_train, 10)
y_train.shape

(28140, 10)

In [15]:
y_valid = to_categorical(y_valid, 10)
y_valid.shape

(13860, 10)

In [16]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               409856    
__________

In [17]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

In [18]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [19]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [20]:
model.fit_generator(datagen.flow(x_train,y_train, batch_size=86), 
                    epochs=1, verbose=1, validation_data=(x_valid, y_valid),
                    steps_per_epoch=x_train.shape[0],
                   callbacks=[learning_rate_reduction])

Epoch 1/1
12449/28140 [============>.................] - ETA: 8:50 - loss: 0.1094 - acc: 0.9682

In [21]:
x_pred = model.predict(x_test)
x_pred.shape

(28000, 10)

In [22]:
x_classes=x_pred.argmax(axis=-1)
x_classes.shape

(28000,)

In [23]:
results = Series(x_classes,name="Label")

In [24]:
submission = concat([Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("digit_recog_cnn.csv",index=False)